In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('GFM_data.csv', sep = '\t')

In [2]:
import tensorflow as tf # build model
import string # get set of punctuations
import requests # get data file in notebook

In [3]:
#!pip install nltk
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#nltk.download('wordnet')
#nltk.download('stopwords')
from nltk.corpus import wordnet

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
REPLACE_IP_ADDRESS = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')
def extract_entities(text):
    names = []
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label'):
                names.append(' '.join(c[0] for c in chunk.leaves()))
    new_text = text
    for name in names:
        if name in text:
            new_text = new_text.replace(name, 'NLP')
    return new_text

def clean_text(x):
    ## removing names
    x = extract_entities(x)
    ## normalizing text by stripping white space and lower casing
    x =  x.lower().strip()
    ## removing urls
    x = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', x)
    ## removing phone numbers
    x = re.sub('\([0-9]{3}\)\s*[0-9]{3}-[0-9]{4}','',x)
    ## strip all non alphanumeric things
    x = re.sub('\n',' ',x)
    x = re.sub("[^a-zA-Z0-9 #]",'',x)
    x = re.sub("\s+",' ',x)
    text = x.replace('\n', ' ').lower()# lowercase text
    text = REPLACE_IP_ADDRESS.sub('', text) # remove ip address
    text = REPLACE_BY_SPACE_RE.sub(' ',text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('',text)# delete symbols which are in BAD_SYMBOLS_RE from text
    # originally:
    # text = ' '.join([w for w in text.split() if not w in STOPWORDS])# delete stopwords from text
    text = [w for w in text.split() if not w in STOPWORDS]# delete stopwords from text   
    return text

In [4]:
no_nan = df.dropna()
no_nan['Text'] = no_nan['Text'].apply(clean_text)
text = no_nan.get('Text').to_numpy()

<ipython-input-4-fc72f8768162>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_nan['Text'] = no_nan['Text'].apply(clean_text)


In [24]:
#text # check what's inside

In [6]:
tokens = [] # combine words
for lst in text:
    tokens += lst
len(tokens)

144109

In [7]:
#text = ' '.join(text) # join all array element
#new_text = clean_text(text)
#new_text
length = 51 # learn initial 50 words, predict the next word
lines = [] # lst of all training sequences
tokens = text[0]
sufficiency = 200_000
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    lines.append(line)
    if i > sufficiency:
        break

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import LambdaCallback

In [9]:
tokenizer = Tokenizer(oov_token="<OOV>") # OOV adds new words outside the trained words
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines) # all words become numbers

In [10]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1] # 50 first words in X, the last word in y

In [11]:
# one-hot encoding
vocab_size = len(tokenizer.word_index) + 1
y = to_categorical(y, num_classes=vocab_size)

In [12]:
sequence_length = len(X[0]) # Length of each list in X is 50 (might also use X.shape[1])
 
# LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length = sequence_length)) # input dim - vocabsize, output dim = 50
# 100 hidden layers
model.add(LSTM(100, return_sequences=True)) # do twice, return_sequences=True
model.add(LSTM(100)) 
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            4100      
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 82)                8282      
Total params: 163,282
Trainable params: 163,282
Non-trainable params: 0
_________________________________________________________________


In [14]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    text = []
    
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

        y_predict = model.predict_classes(encoded)

        predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == y_predict:
                predicted_word = word
                break
        seed_text = seed_text + ' ' + predicted_word
        text.append(predicted_word)
    return ' '.join(text)

In [26]:
alltext = df['Text'].str.cat(sep='. ').lower()
#alltext

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
model.fit(X, y, batch_size=256, epochs=100)

Epoch 1/100
1/1 [==============================] - 0s 74ms/step - loss: 1.5574 - accuracy: 0.5610
Epoch 2/100
1/1 [==============================] - 0s 62ms/step - loss: 1.5322 - accuracy: 0.6098
Epoch 3/100
1/1 [==============================] - 0s 56ms/step - loss: 1.5373 - accuracy: 0.6585
Epoch 4/100
1/1 [==============================] - 0s 57ms/step - loss: 1.5172 - accuracy: 0.6098
Epoch 5/100
1/1 [==============================] - 0s 67ms/step - loss: 1.4922 - accuracy: 0.6341
Epoch 6/100
1/1 [==============================] - 0s 62ms/step - loss: 1.4543 - accuracy: 0.8049
Epoch 7/100
1/1 [==============================] - 0s 62ms/step - loss: 1.4401 - accuracy: 0.8049
Epoch 8/100
1/1 [==============================] - 0s 59ms/step - loss: 1.4202 - accuracy: 0.8049
Epoch 9/100
1/1 [==============================] - 0s 57ms/step - loss: 1.4250 - accuracy: 0.6829
Epoch 10/100
1/1 [==============================] - 0s 61ms/step - loss: 1.4313 - accuracy: 0.6341
Epoch 11/100
1/1 [=

1/1 [==============================] - 0s 96ms/step - loss: 0.6472 - accuracy: 0.9756
Epoch 84/100
1/1 [==============================] - 0s 92ms/step - loss: 0.6570 - accuracy: 0.9756
Epoch 85/100
1/1 [==============================] - 0s 101ms/step - loss: 0.6440 - accuracy: 0.9756
Epoch 86/100
1/1 [==============================] - 0s 101ms/step - loss: 0.6294 - accuracy: 0.9756
Epoch 87/100
1/1 [==============================] - 0s 101ms/step - loss: 0.6452 - accuracy: 0.9756
Epoch 88/100
1/1 [==============================] - 0s 106ms/step - loss: 0.6337 - accuracy: 0.9756
Epoch 89/100
1/1 [==============================] - 0s 113ms/step - loss: 0.6471 - accuracy: 0.9756
Epoch 90/100
1/1 [==============================] - 0s 113ms/step - loss: 0.6768 - accuracy: 0.9024
Epoch 91/100
1/1 [==============================] - 0s 105ms/step - loss: 0.6999 - accuracy: 0.9512
Epoch 92/100
1/1 [==============================] - 0s 112ms/step - loss: 0.7587 - accuracy: 0.8293
Epoch 93/100
1/

In [96]:
seed_text=alltext[:len(df['Text'][0])+16]
seq_length = len(X[0])
#seed_text

In [97]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    text = []

    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

        y_predict = model.predict_classes(encoded)

        predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == y_predict:
                predicted_word = word
                break
        print('--SEED TEXT--')
        print(seed_text)
        seed_text = seed_text + ' ' + predicted_word
        print('--PREDICTED WORDS--', predicted_word)
        print('----------------------------------------------------------')
        text.append(predicted_word)
    return ' '.join(text)

In [98]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo na
--PREDICTED W

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo na act violence 

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo na act violence 

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo na act violence 

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo na act violence 

...

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo na act violence 

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo na act violence 

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo na act violence 

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo na act violence 

'act violence anyone anyone wishing send cards cards cards encouragement encouragement andor contributions form money order check may may mail nlp nlp nlp 122 calhoun street tallahassee fl 32301 attn attn justice justice portion portion proceeds proceeds proceeds also used used benefit sons six six six children witnessed horrific horrific act act anyone wishing cards cards encouragement andor contributions form order check may mail nlp nlp 122 calhoun street fl 32301 32301 attn attn continue justice justice justice justice portion portion proceeds proceeds proceeds also also used used benefit sons six six children witnessed horrific act act anyone wishing send'

https://kgptalkie.com/text-generation-using-tensorflow-keras-and-lstm/

https://www.youtube.com/watch?v=VAMKuRAh2nc&t=1607s&ab_channel=KGPTalkie